In [1]:
import json

def find_opinion_by_lexicon(sens, lexicon_file, rate):
    find_opinion = []
    score_dic = {}
    with open(lexicon_file) as file_obj:
        pri_list = json.load(file_obj)
    for i, s in enumerate(sens):
        score = 0
        for ss in s:
            if ss.lower() in pri_list or ss in pri_list:
                score += 1
            if ss.lower() in words_md:
                score += 2
        score_dic[i] = score
    
    sorted_dic = sorted(score_dic.items(), key=lambda item:item[1], reverse=True)
    extract_length = int(len(sens)*rate)
    for i, sd in enumerate(sorted_dic):
        if i < extract_length:
            find_opinion.append(sens[sd[0]])
    return find_opinion

In [2]:
# load train set
opinions = []
for i in range(2, 15):
    with open(str(i)+'_batch/opinion_token.json') as file_obj:
        opinion = json.load(file_obj)
        opinions.extend(opinion)
    
with open('1_batch/driver_token.json') as file_obj:
    opinion = json.load(file_obj)
    opinions.extend(opinion) 
    
with open('1_batch/barrier_token.json') as file_obj:
    opinion = json.load(file_obj)
    opinions.extend(opinion)

print(len(opinions))

1237


In [3]:
import import_ipynb
from utils import construct_dict
import os

if 'opinion_dic.json' in os.listdir():
    with open('opinion_dic.json') as file_obj:
        opinion_dic = json.load(file_obj)
else:
    opinion_dic = construct_dict(opinions)
    
if 'lexicon_dic.json' in os.listdir():
    with open('lexicon_dic.json') as file_obj:
        lexicon_dic = json.load(file_obj)
else:
    print('please run unsupervised_lexicon.ipynb first to get lexicon_dic.json')

importing Jupyter notebook from utils.ipynb


In [4]:
import numpy as np
from sklearn import preprocessing

opinion_words = []
opinion_freq = []
lexicon_freq = []
words_md = ['could', 'may', 'would', 'must', 'might', 'shall', 'ought', 'can']
for od in opinion_dic.keys():
    if od in lexicon_dic.keys():
        opinion_words.append(od)
        opinion_freq.append(opinion_dic[od])
        lexicon_freq.append(lexicon_dic[od])
print(len(lexicon_dic.keys()), 'words in lexicon_dic')
print(len(opinion_dic.keys()), 'words in opinion_dic')

12889 words in lexicon_dic
3205 words in opinion_dic


In [5]:
# Sort(Normalization(Opinion_freq) - Normalization(lexicon_freq)) to construct dictionary

opinion_freq = np.array(opinion_freq).reshape(1, -1)
normalizer_opinion = preprocessing.Normalizer().fit_transform(opinion_freq)
normalizer_opinion = list(normalizer_opinion[0])

lexicon_freq = np.array(lexicon_freq).reshape(1, -1)
normalizer_lexicon = preprocessing.Normalizer().fit_transform(lexicon_freq)
normalizer_lexicon = list(normalizer_lexicon[0])

more_value = {}
for i, w in enumerate(opinion_words):
    more_value[w] = normalizer_opinion[i]-normalizer_lexicon[i]
pri_words = sorted(more_value.items(), key=lambda item:item[1], reverse=True)
pri_list_whole = []
i = 0
for p_set in pri_words:
    if p_set[0] not in words_md and opinion_dic[p_set[0]]>1:
        # print(i, p_set, opinion_dic[p_set[0]])
        pri_list_whole.append(p_set[0])
        i += 1
        
# The amount can be adjusted, here choose 100  
pri_list = pri_list_whole[:100]
# json_str = json.dumps(pri_list, indent=4)
# with open('pri_200.json', 'w', encoding='utf-8') as json_file:
    # json_file.write(json_str)
print(pri_list)

['recycling', 'incentive', 'product', 'EoL', 'scheme', 'collection', 'management', 'policy', 'industry', 'design', 'lack', 'economic', 'responsibility', 'regulation', 'waste', 'producer', 'government', 'business', 'recyclers', 'material', 'consumer', 'environmental', 'infrastructure', 'manufacturer', 'recycle', 'awareness', 'recovery', 'disposal', 'landfill', 'stewardship', 'stakeholder', 'raw', 'create', 'reduce', 'appropriate', 'Regulations', 'develop', 'improve', 'ensure', 'encourage', 'activity', 'economically', 'challenge', 'valuable', 'opportunity', 'benefit', 'new', 'sustainable', 'essential', 'hazardous', 'Manufacturers', 'need', 'efficient', 'avoid', 'effective', 'profitable', 'cost', 'long', 'circular', 'loop', 'issue', 'party', 'still', 'burden', 'environmentally', 'target', 'EOL', 'effort', 'make', 'proper', 'developing', 'reducing', 'promote', 'necessary', 'social', 'future', 'fully', 'uncertainty', 'viability', 'financial', 'introducing', 'job', 'robust', 'rare', 'investm

# Load test set (can be replaced to other set/file)

In [46]:
sens = []
opinion_answer = []
driver_answer = []
barrier_answer = []

with open('UNSW_answer_without_conclusion/sen_token.json') as file_obj:
    sen = json.load(file_obj)
    sens.extend(sen)
with open('UNSW_answer_without_conclusion/driver_token.json') as file_obj:
    driver = json.load(file_obj)
    driver_answer.extend(driver)
    opinion_answer.extend(driver)
with open('UNSW_answer_without_conclusion/barrier_token.json') as file_obj:
    barrier = json.load(file_obj)
    barrier_answer.extend(barrier)
    opinion_answer.extend(barrier)
with open('UNSW_answer_2/sen_token.json') as file_obj:
    sen = json.load(file_obj)
    sens.extend(sen)
with open('UNSW_answer_2/driver_token.json') as file_obj:
    driver = json.load(file_obj)
    driver_answer.extend(driver)
    opinion_answer.extend(driver)
with open('UNSW_answer_2/barrier_token.json') as file_obj:
    barrier = json.load(file_obj)
    barrier_answer.extend(barrier)
    opinion_answer.extend(barrier)
    
print(str(len(sens)) + ' sentences in total')
print(str(len(opinion_answer)) + ' are opinion')
print(len(opinion_answer)/len(sens))

695 sentences in total
153 are opinion
0.22014388489208633


In [75]:
find_opinion = find_opinion_by_lexicon(sens, 'pri_100.json', 0.2)
print(len(find_opinion))

139


In [76]:
from utils import calculate_metric

print('Precision, Recall, F1_Score, Accuracy')
print(calculate_metric(find_opinion, opinion_answer, len(sens)))

Precision, Recall, F1_Score, Accuracy
(0.4460431654676259, 0.40522875816993464, 0.4246575342465754, 0.758273381294964)
